In [66]:
from fatiando.gravmag import normal_gravity
from fatiando.vis import mpl
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.basemap import Basemap, shiftgrid, cm
import urllib

In [67]:
lon, lat, height, gravity = np.loadtxt('eigen-6c4-grav.gdf', skiprows=34,
                                       unpack=True)
topo = np.loadtxt('eigen-6c4-topo.gdf', skiprows=29, usecols=[-1], unpack=True)
shape = (53, 210)

area = (lon.min(), lon.max(), lat.min(), lat.max())

In [68]:
# First, lets calculate the gravity disturbance (e.g., the free-air anomaly)
# We'll do this using the closed form of the normal gravity for the WGS84
# ellipsoid
gamma = normal_gravity.gamma_closed_form(lat, height)
disturbance = gravity - gamma

In [69]:
# Now we can remove the effect of the Bouguer plate to obtain the Bouguer
# anomaly. We'll use the standard densities of 2.67 g.cm^-3 for crust and 1.04
# g.cm^-3 for water.
bouguer = disturbance - normal_gravity.bouguer_plate(topo)

In [5]:
#mpl.figure(figsize=(14, 3.5))
mpl.figure(figsize=(28, 7))
bm = mpl.basemap(area, projection='merc')

mpl.subplot(131)
mpl.title('Gravity (mGal)')
mpl.contourf(lon, lat, gravity, shape, 60, cmap=mpl.cm.Reds, basemap=bm)
mpl.colorbar(pad=0.05)

mpl.subplot(132)
mpl.title('Gravity disturbance (mGal)')
amp = np.abs(disturbance).max()
mpl.contourf(lon, lat, disturbance, shape, 60, cmap=mpl.cm.RdBu_r, basemap=bm,
             vmin=-amp, vmax=amp)
mpl.colorbar(pad=0.05)

mpl.subplot(133)
mpl.title('Bouguer anomaly (mGal)')
mpl.contourf(lon, lat, bouguer, shape, 60, cmap=mpl.cm.Reds, basemap=bm)
mpl.colorbar(pad=0.05)
mpl.show()

In [74]:
import numpy as np
def _bkmatrix(x,y, degree):
    """
    Make the Bk polynomial coefficient matrix for a given PointGrid.
    This matrix converts the coefficients into physical property values.
    Parameters:
    * grid : :class:`~fatiando.mesher.PointGrid`
        The sources in the equivalent layer
    * degree : int
        The degree of the bivariate polynomial
    Returns:
    * bk : 2d-array
        The matrix
    Examples:
    >>> from fatiando.mesher import PointGrid
    >>> grid = PointGrid((0, 1, 0, 2), 10, (2, 2))
    >>> print _bkmatrix(grid, 2)
    [[ 1.  0.  0.  0.  0.  0.]
     [ 1.  0.  1.  0.  0.  1.]
     [ 1.  2.  0.  4.  0.  0.]
     [ 1.  2.  1.  4.  2.  1.]]
    >>> print _bkmatrix(grid, 1)
    [[ 1.  0.  0.]
     [ 1.  0.  1.]
     [ 1.  2.  0.]
     [ 1.  2.  1.]]
    >>> print _bkmatrix(grid, 3)
    [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
     [ 1.  0.  1.  0.  0.  1.  0.  0.  0.  1.]
     [ 1.  2.  0.  4.  0.  0.  8.  0.  0.  0.]
     [ 1.  2.  1.  4.  2.  1.  8.  4.  2.  1.]]
    """
    bmatrix = np.transpose(
        [(x ** i) * (y ** j)
         for l in xrange(1, degree + 2)
         for i, j in zip(xrange(l), xrange(l - 1, -1, -1))])
    return bmatrix

In [75]:
A_bouguer=_bkmatrix(lon,lat,3)

In [76]:
A_bouguer.shape

(11130, 10)

In [77]:
w=1.
W=np.identity(A_bouguer.shape[0])
rwlst_it=10
pw=0
epsilon=0.0001
for i in range(rwlst_it):
    #lstsq
    p_bouguer = np.dot(A_bouguer.transpose(),W)
    p_bouguer = np.linalg.inv(np.dot(p_bouguer,A_bouguer))    
    p_bouguer = np.dot(p_bouguer,A_bouguer.transpose())
    p_bouguer = np.dot(p_bouguer,w*bouguer)
    #
    r = bouguer-np.dot(A_bouguer,p_bouguer)
    w = 1/np.abs(r)
    W=np.diag(w)
    print 'i=',i,'\n'
    #print 'p=',pw,'\n','W=',W,'\n','mean(r)=',np.mean(r),'\n','--------------------'
#pw

i= 0 

i= 1 

i= 2 

i= 3 

i= 4 

i= 5 

i= 6 

i= 7 

i= 8 

i= 9 



In [78]:
np.mean(np.abs(bouguer-np.dot(A_bouguer,p_bouguer)))

13.738698448806474

In [79]:
grav_corr=bouguer-np.dot(A_bouguer,p_bouguer)

In [30]:
mpl.figure()
mpl.title('Gravity (mGal)')
mpl.contourf(lon, lat, gravity, shape, 60, cmap=mpl.cm.RdBu_r, basemap=bm)
#mpl.contourf(lon, lat, gravity, shape, 60, cmap=mpl.cm.spectral, basemap=bm)
mpl.colorbar(pad=0.05)
mpl.show()

In [29]:
mpl.figure()
mpl.title('Gravity free air (mGal)')
amp = np.abs(disturbance).max()
mpl.contourf(lon, lat, disturbance, shape, 60, cmap=mpl.cm.RdBu_r, basemap=bm,
             vmin=-amp, vmax=amp)
mpl.colorbar(pad=0.05)
mpl.show()

In [31]:
mpl.figure()
mpl.title('Bouguer anomaly (mGal)')
mpl.contourf(lon, lat, bouguer, shape, 60, cmap=mpl.cm.RdBu_r, basemap=bm)
#mpl.contourf(lon, lat, bouguer, shape, 60, cmap=mpl.cm.hsv, basemap=bm)
mpl.colorbar(pad=0.05)
mpl.show()

In [32]:
plt.figure()
plt.title('Gravidade residual (mGal)')
mpl.contourf(lon, lat, grav_corr, shape, 60, cmap=mpl.cm.RdBu_r, basemap=bm)
#mpl.contourf(lon, lat, grav_corr, shape, 60, cmap=mpl.cm.hsv, basemap=bm)
plt.colorbar(pad=0.05)
plt.show()

In [22]:
np.savetxt('bouguer.txt',np.transpose([lon, lat, bouguer]), fmt='%1.12f')

In [23]:
np.savetxt("gravity_free_air.txt",np.transpose([lon, lat, disturbance]), fmt='%1.12f')

In [24]:
np.savetxt("gravity.txt",np.transpose([lon, lat, gravity]), fmt='%1.12f')

In [25]:
np.savetxt("grav_residual.txt",np.transpose([lon, lat, grav_corr]), fmt='%1.12f') 

# Mapas

In [80]:
lon_tmp = lon
lat_tmp = lat
topo_tmp = topo
topo = topo.reshape([53,210])
lon = lon.reshape([53,210])
lat = lat.reshape([53,210])

In [81]:
lon = -360.+lon

### Mapa topografia

In [139]:
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
m = Basemap(llcrnrlon=lon.min(), llcrnrlat=lat.min(), urcrnrlon=lon.max(), urcrnrlat=lat.max(),
            resolution='l', projection='merc',
            lat_1=lat.max(), lon_0=lon.max())
x, y = m(lon, lat)
im=m.contourf(x, y, topo, 40, cmap='cubehelix')
parallels = np.arange(-21.5,-19,0.5)
m.drawcoastlines()
m.drawcountries()
#parallels
parallels = np.arange(-21.5,-19,0.5)
m.drawparallels(parallels,labels=[1,0,0,1])
##meridians
meridians = np.arange(320.,331.,2.5)
m.drawmeridians(meridians,labels=[1,0,0,1])
#
cb = m.colorbar(im,"right", size="5%", pad='5%')
cb.ax.set_xlabel('m')
ax.set_title('Topografia')
plt.show()

### Mapa para valores de gravidade

In [83]:
grav_tmp=gravity.reshape([53,210])

In [140]:
#plt.figure()
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
m = Basemap(llcrnrlon=lon.min(), llcrnrlat=lat.min(), urcrnrlon=lon.max(), urcrnrlat=lat.max(),
            resolution='l', projection='merc',
            lat_1=lat.max(), lon_0=lon.max())
x, y = m(lon, lat)
im=m.contourf(x, y, grav_tmp, 100, cmap='cubehelix')
parallels = np.arange(-21.5,-19,0.5)
m.drawcoastlines()
m.drawcountries()
#parallels
parallels = np.arange(-21.5,-19,0.5)
m.drawparallels(parallels,labels=[1,0,0,1])
#meridians
meridians = np.arange(320.,331.,2.5)
m.drawmeridians(meridians,labels=[1,0,0,1])
#
cb = m.colorbar(im,"right", size="5%", pad='5%')
cb.ax.set_xlabel('mgal')
ax.set_title(u'Aceleração da gravidade')
plt.show()

### Mapa anomalia ar livre

In [85]:
disturbance_tmp=disturbance.reshape([53,210])

In [88]:
maior=np.abs([disturbance_tmp.min(), disturbance_tmp.max()]).max()

In [141]:
#plt.figure()
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
m = Basemap(llcrnrlon=lon.min(), llcrnrlat=lat.min(), urcrnrlon=lon.max(), urcrnrlat=lat.max(),
            resolution='l', projection='merc',
            lat_1=lat.max(), lon_0=lon.max())
x, y = m(lon, lat)
im=m.contourf(x, y, disturbance_tmp, 100, cmap='RdBu_r', vmin=-maior, vmax=maior)
#parallels
parallels = np.arange(-21.5,-19,0.5)
m.drawparallels(parallels,labels=[1,0,0,1])
#meridians
meridians = np.arange(320.,331.,2.5)
m.drawmeridians(meridians,labels=[1,0,0,1])
#
cb = m.colorbar(im,"right", size="5%", pad='5%')
cb.ax.set_xlabel('mgal')
ax.set_title('Anomalia ar-livre')
plt.show()

### Mapa anomalia bouguer

In [97]:
bouguer_tmp=bouguer.reshape([53,210])
maior=np.abs([bouguer_tmp.min(), bouguer_tmp.max()]).max()

In [143]:
#plt.figure()
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
m = Basemap(llcrnrlon=lon.min(), llcrnrlat=lat.min(), urcrnrlon=lon.max(), urcrnrlat=lat.max(),
            resolution='l', projection='merc',
            lat_1=lat.max(), lon_0=lon.max())
x, y = m(lon, lat)
im=m.contourf(x, y, bouguer_tmp, 100, cmap='cubehelix')
#parallels
parallels = np.arange(-21.5,-19,0.5)
m.drawparallels(parallels,labels=[1,0,0,1])
#meridians
meridians = np.arange(320.,331.,2.5)
m.drawmeridians(meridians,labels=[1,0,0,1])
#
cb = m.colorbar(im,"right", size="5%", pad='5%')
cb.ax.set_xlabel('mgal')
ax.set_title('Anomalia Bouguer')
plt.show()

In [106]:
bouguer_tmp.max()

383.63963183468326

### Mapa Anomalia Residual da Gravidade

In [113]:
grav_corr_tmp=grav_corr.reshape([53,210])
maior=np.abs([grav_corr_tmp.min(), grav_corr_tmp.max()]).max()

In [119]:
grav_corr_tmp.max()

51.122305023340573

In [145]:
#plt.figure()
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
m = Basemap(llcrnrlon=lon.min(), llcrnrlat=lat.min(), urcrnrlon=lon.max(), urcrnrlat=lat.max(),
            resolution='l', projection='merc',
            lat_1=lat.max(), lon_0=lon.max())
x, y = m(lon, lat)
im=m.contourf(x, y, grav_corr_tmp, 50, cmap='RdBu_r',vmin=-maior,vmax=maior)
#parallels
parallels = np.arange(-21.5,-19,0.5)
m.drawparallels(parallels,labels=[1,0,0,1])
#meridians
meridians = np.arange(320.,331.,2.5)
m.drawmeridians(meridians,labels=[1,0,0,1])
#
cb = m.colorbar(im,"right", size="5%", pad='5%')
cb.ax.set_xlabel('mgal')
ax.set_title('Anomalia Residual da Gravidade')
plt.show()

# Load ICGEM data - Leo function

In [38]:
grav_data=load_icgem_gdf('eigen-6c4-grav.gdf')

In [39]:
grav_data

{'area': [-21.887758, -19.287758, 320.988022, 331.438022],
 'gravity_earth': array([ 978724.5973651 ,  978720.86597657,  978721.55250247, ...,
         978591.0472444 ,  978591.47316492,  978591.68283901]),
 'h_over_geoid': array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 'latitude': array([-21.8878, -21.8878, -21.8878, ..., -19.2878, -19.2878, -19.2878]),
 'longitude': array([ 320.988,  321.038,  321.088, ...,  331.338,  331.388,  331.438]),
 'metadata': 'generating_institute     gfz-potsdam\n     generating_date     2015/09/15\n        product_type     gravity_field\n                body     earth\n           modelname     eigen-6c4\n     max_used_degree          2190\n         tide_system     tide_free\n          functional     gravity_earth  (centrifugal term included)\n                unit     mgal\n          refsysname     WGS84\n            gmrefpot      3.98600441800E+14 m**3/s**2\n        radiusrefpot     6378137.000 m\n          flatrefpot      3.352810664747480E-03   (1/298.2572

In [37]:
def load_icgem_gdf(fname, usecols=None):
    """
    Load data from an ICGEM .gdf file.
    
    Returns:
    
    * data : dict
        A dictionary with the data from the file. 
        Reads the column data and other metadata from 
        the file. Column data are numpy arrays.
        
    """
    with open(fname) as f:
        # Read the header and extract metadata
        metadata = []
        shape = [None, None]
        size = None
        height = None
        attributes = None
        attr_line = False
        area = [None]*4
        for line in f:
            if line.strip()[:11] == 'end_of_head':
                break
            metadata.append(line)
            if not line.strip():
                attr_line = True
                continue
            if not attr_line:
                parts = line.strip().split()
                if parts[0] == 'height_over_ell':
                    height = float(parts[1])
                elif parts[0] == 'latitude_parallels':
                    shape[0] = int(parts[1])
                elif parts[0] == 'longitude_parallels':
                    shape[1] = int(parts[1])
                elif parts[0] == 'number_of_gridpoints':
                    size = int(parts[1])
                elif parts[0] == 'latlimit_south':
                    area[0] = float(parts[1])
                elif parts[0] == 'latlimit_north':
                    area[1] = float(parts[1])
                elif parts[0] == 'longlimit_west':
                    area[2] = float(parts[1])
                elif parts[0] == 'longlimit_east':
                    area[3] = float(parts[1])
            else:
                attributes = line.strip().split()
                attr_line = False
        # Read the numerical values
        rawdata = np.loadtxt(f, usecols=usecols, ndmin=2, unpack=True)
    # Sanity checks
    assert all(n is not None for n in shape), "Couldn't read shape of grid."
    assert size is not None, "Couldn't read size of grid."
    shape = tuple(shape)
    assert shape[0]*shape[1] == size, \
        "Grid shape '{}' and size '{}' mismatch.".format(shape, size)
    assert attributes is not None, "Couldn't read column names."
    if usecols is not None:
        attributes = [attributes[i] for i in usecols]
    assert len(attributes) == rawdata.shape[0], \
        "Number of attributes ({}) and data columns ({}) mismatch".format(
            len(attributes), rawdata.shape[0])
    assert all(i is not None for i in area), "Couldn't read the grid area."
    # Return the data in a dictionary with the attribute names
    # that we got from the file.
    data = dict(shape=shape, area=area, metadata=''.join(metadata))
    for attr, value in zip(attributes, rawdata):
        # Need to invert the data matrices in latitude "[::-1]"
        # because the ICGEM grid gets varies latitude from N to S
        # and the TesseroidRelief expects the opposite.
        data[attr] = value.reshape(shape)[::-1].ravel()
    if (height is not None) and ('height' not in attributes):
        data['height'] = height*np.ones(size)
    if 'latitude' in attributes and 'longitude' in attributes:
        lat, lon = data['latitude'], data['longitude']
        area = (lat.min(), lat.max(), lon.min(), lon.max())
        assert np.allclose(area, data['area']), \
            "Grid area read ({}) and calculated from attributes ({}) mismatch.".format(
                data['area'], area)
    return data 